In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

import pandas as pd
import cv2
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import PolynomialFeatures, normalize, StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, classification_report, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV

from PIL import Image

import time

In [2]:
# Loading Data
data_train = np.load('data_train.npy')
labels_train = np.load('labels_train.npy')

print(data_train.shape, labels_train.shape)

(270000, 3717) (3717,)


In [3]:
# Loading Unknown Class Data
data_unknown = np.load('data_unknown.npy')
labels_unknown = np.load('labels_unknown.npy')

print(data_unknown.shape, labels_unknown.shape)

(270000, 2000) (2000,)


In [4]:
data_train = np.concatenate((data_train, data_unknown),axis = 1)
labels_train = np.concatenate((labels_train, labels_unknown))
labels_train[labels_train == -1] = 10

In [5]:
X = data_train.T
t = labels_train
#orig = X[0,:].reshape((300,300,3))

In [6]:
'''def resizeImg(image, output_size):
    input_size = image.shape[0]
    output_size = output_size
    bin_size = input_size // output_size
    small_image = image.reshape((output_size, bin_size, 
                                       output_size, bin_size)).max(3).max(1)
    #small_image = image.reshape((output_size, bin_size, 
    #                                   output_size, bin_size, 3)).max(3).max(1)
    return small_image

def grayConversion(image):
    grayValue = 0.07 * image[:,:,2] + 0.72 * image[:,:,1] + 0.21 * image[:,:,0]
    gray_img = grayValue.astype(np.uint8)
    return gray_img

#g = grayConversion(orig)

X_gray = np.zeros((labels_train.shape[0],90000))
for i in range(X.shape[0]):
    gray_img = grayConversion(X[i,:].reshape((300,300,3)))
    X_gray[i,:] = gray_img.ravel()[:,np.newaxis].T

output_size = 50
X_small = np.zeros((data_train.shape[0],(output_size**2)))
for i in range(X.shape[0]):
    small_img = resizeImg(X[i,:].reshape((300,300,3)), output_size)
    X_small[i,:] = small_img.ravel()[:,np.newaxis].T

for i in range(X.shape[0]):
    small_img = resizeImg(X_gray[i,:].reshape((300,300)), output_size)
    X_small[i,:] = small_img.ravel()[:,np.newaxis].T
    
X_small.shape'''

'def resizeImg(image, output_size):\n    input_size = image.shape[0]\n    output_size = output_size\n    bin_size = input_size // output_size\n    small_image = image.reshape((output_size, bin_size, \n                                       output_size, bin_size)).max(3).max(1)\n    #small_image = image.reshape((output_size, bin_size, \n    #                                   output_size, bin_size, 3)).max(3).max(1)\n    return small_image\n\ndef grayConversion(image):\n    grayValue = 0.07 * image[:,:,2] + 0.72 * image[:,:,1] + 0.21 * image[:,:,0]\n    gray_img = grayValue.astype(np.uint8)\n    return gray_img\n\n#g = grayConversion(orig)\n\nX_gray = np.zeros((labels_train.shape[0],90000))\nfor i in range(X.shape[0]):\n    gray_img = grayConversion(X[i,:].reshape((300,300,3)))\n    X_gray[i,:] = gray_img.ravel()[:,np.newaxis].T\n\noutput_size = 50\nX_small = np.zeros((data_train.shape[0],(output_size**2)))\nfor i in range(X.shape[0]):\n    small_img = resizeImg(X[i,:].reshape((300,300,

In [7]:
def resize_and_convert_grayscale(np_image, new_size):
    # Convert NumPy array to PIL Image
    img = Image.fromarray(np_image)
    
    # Convert the image to grayscale
    img_gray = img.convert('L')
    
    # Resize the image
    img_resized = img_gray.resize(new_size, Image.ANTIALIAS)
    
    # Convert PIL Image back to NumPy array
    np_resized = np.array(img_resized)

    return np_resized

# Usage
input_images = X
original_shape = (300, 300, 3) 
new_size = (50, 50) 

# Initialize an empty list to store the processed images
processed_images = []

for i in range(input_images.shape[0]):
    # Reshape the 1D flattened image back to its original 2D or 3D shape
    image_2D = input_images[i].reshape(original_shape)

    # Apply resizing and grayscale conversion
    output_image = resize_and_convert_grayscale(image_2D, new_size)

    # Flatten the output image and add it to the list of processed images
    processed_images.append(output_image.flatten())

# Convert the list of processed images back to a 2D NumPy array
output_images = np.array(processed_images)

/scratch/local/62545500/ipykernel_127157/1684613638.py:9: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = img_gray.resize(new_size, Image.ANTIALIAS)


In [8]:
output_images.shape

(5717, 2500)

In [9]:
start = time.time()

x_train, x_test, t_train, t_test = train_test_split(output_images, t, test_size=0.2, random_state=0)
svc_rbf = Pipeline([('scaler', StandardScaler()),
               ('SVC', SVC(kernel='rbf'))])

# soft-margin SVM with RBF kernel
parameters = {
    'SVC__gamma': ['scale', 'auto'],
    'SVC__C': [1, 5, 10]
}
cv = GridSearchCV(svc_rbf, param_grid=parameters)
cv.fit(x_train, t_train)
model_best = cv.best_estimator_
t_pred_svc_rbf = model_best.predict(x_test)

end = time.time()
elapsed_time = end - start
print(f"Elapsed time: {elapsed_time} s")

Elapsed time: 902.4659230709076 s


In [10]:
model_best

Pipeline(steps=[('scaler', StandardScaler()), ('SVC', SVC(C=5))])

In [11]:
def performance(t_test, t_pred):
    print('Accuracy: \n',accuracy_score(t_test, t_pred))
    print('f1 score: \n',f1_score(t_test, t_pred, average = None))
    print('Classification Report: \n', classification_report(t_test, t_pred))
    print('Confusion Matrix: \n',confusion_matrix(t_test, t_pred))

In [12]:
t_pred_train = model_best.predict(x_train)
print('Soft-margin SVM with RBF kernel Performance in Train Set:')
performance(t_train, t_pred_train)

Soft-margin SVM with RBF kernel Performance in Train Set:
Accuracy: 
 0.9606385305051388
f1 score: 
 [0.93451327 0.95532646 0.98245614 0.98671096 0.98615917 0.97315436
 0.94454383 0.98648649 0.96949153 0.93428064 0.95192595]
Classification Report: 
               precision    recall  f1-score   support

         0.0       0.99      0.88      0.93       299
         1.0       1.00      0.91      0.96       304
         2.0       1.00      0.97      0.98       290
         3.0       1.00      0.97      0.99       305
         4.0       1.00      0.97      0.99       293
         5.0       0.99      0.95      0.97       304
         6.0       0.99      0.90      0.94       293
         7.0       1.00      0.97      0.99       300
         8.0       0.99      0.95      0.97       300
         9.0       0.97      0.90      0.93       291
        10.0       0.91      1.00      0.95      1594

    accuracy                           0.96      4573
   macro avg       0.99      0.94      0.96   

In [13]:
print('Soft-margin SVM with RBF kernel Performance in Test Set:')
performance(t_test, t_pred_svc_rbf)

Soft-margin SVM with RBF kernel Performance in Test Set:
Accuracy: 
 0.4982517482517482
f1 score: 
 [0.1682243  0.30769231 0.34586466 0.37931034 0.58267717 0.416
 0.38983051 0.33333333 0.26415094 0.30534351 0.65221319]
Classification Report: 
               precision    recall  f1-score   support

         0.0       0.30      0.12      0.17        77
         1.0       0.44      0.24      0.31        68
         2.0       0.45      0.28      0.35        82
         3.0       0.44      0.33      0.38        66
         4.0       0.77      0.47      0.58        79
         5.0       0.47      0.37      0.42        70
         6.0       0.50      0.32      0.39        72
         7.0       0.40      0.29      0.33        66
         8.0       0.42      0.19      0.26        73
         9.0       0.43      0.24      0.31        85
        10.0       0.51      0.89      0.65       406

    accuracy                           0.50      1144
   macro avg       0.47      0.34      0.38      114